# Building inverted index and answering queries

In this lab you are going to implement a standard document processing pipeline and then build a simple search engine based on it:
- starting from crawling documents, 
- then building an inverted index,
- and answering queries using this index.

## Preprocessing

First, we need a unified approach to documents and queries preprocessing. Implement a class responsible for that. Complete the code for given functions (most of them are just one-liners) and make sure you pass the tests. Make use of `nltk` library, `spacy`, or any other you know.

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

class Preprocessor:
    
    def __init__(self):
        self.stop_words = { 'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 
                            'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its',
                            'of', 'on', 'that', 'the', 'to', 'was', 
                            'were', 'will', 'with'}
        self.ps = nltk.stem.PorterStemmer()

    
    def tokenize(self, text):
        #TODO word tokenize text using nltk lib
        
        return nltk.word_tokenize(text)

    
    def stem(self, word, stemmer):
        #TODO stem word using provided stemmer

        # YOUR CODE HERE
        return stemmer.stem(word)        
        # return 'stemmed_word'

    
    def is_apt_word(self, word):
        #TODO check if word is appropriate - not a stop word and isalpha, 
        # i.e consists of letters, not punctuation, numbers, dates
        
        return (word not in self.stop_words) and (word.isalpha())

    
    def preprocess(self, text):
        tokenized = self.tokenize(text.lower())
        return [self.stem(w, self.ps) for w in tokenized if self.is_apt_word(w)]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Tests

In [ ]:
prep = Preprocessor()
text = 'To be, or not to be, that is the question'

assert prep.tokenize(text) == ['To', 'be', ',', 'or', 'not', 'to', 'be', 
                               ',', 'that', 'is', 'the', 'question']
assert prep.stem('retrieval', prep.ps) == 'retriev'
assert prep.is_apt_word('qwerty123') is False
assert prep.preprocess(text) == ['or', 'not', 'question']

## Crawling and Indexing

### Base classes

Here are some base classes you will need for writing an indexer. The code is given, still, you need to change it slightly. Namely, the `parse()` method. Add the code which will extract relevant text from the documents, particularly trageting Reuters website structure.

In [ ]:
import requests
from urllib.parse import quote
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.parse
import os
import re

class Document:

    def __init__(self, url):
        self.url = url

    def download(self):
        try:
            response = requests.get(self.url)
            if response.status_code == 200:
                self.content = response.content
                return True
            else:
                return False
        except:
            return False

    def persist(self, path):
        # this code is not supposed to be good :) 
        # Please discuss why this line is bad
        with open(os.path.join(path, quote(self.url).replace('/', '_')), 'wb') as f:
            f.write(self.content)


class HtmlReutersArticle(Document):

    def normalize(self, href):
        if href is not None and href[:4] != 'http':
            href = urllib.parse.urljoin(self.url, href)
        return href

    def parse(self):
        
        def tag_visible(element):
            if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
                return False
            if isinstance(element, Comment):
                return False
            return True
   
        model = BeautifulSoup(self.content, "html.parser")
        
        self.anchors = []
        a = model.find_all('a')
        for anchor in a:
            href = self.normalize(anchor.get('href'))
            text = anchor.text
            self.anchors.append((text, href))
            
        # extract only header and main content
        # discuss why using classes like article-body__content__17Yit 
        # is the wrong strategy
        header = model.find(re.compile('^h1'))
        content = model.find('body')
                        
        if header is None or content is None:
            self.article_text = ""
            return
        # self.header = header
        texts = header.findAll(text=True) + content.findAll(text=True)
        visible_texts = filter(tag_visible, texts)  
        self.article_text = "\n".join(t.strip() for t in visible_texts)

In [ ]:
doc = HtmlReutersArticle("https://www.reuters.com/business/healthcare-pharmaceuticals/amazon-launches-virtual-healthcare-clinic-us-2022-11-15/")
doc.download()
doc.parse()
doc.article_text

<ipython-input-7-f026656eddb7>:67: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  texts = header.findAll(text=True) + content.findAll(text=True)


"RPT Amazon launches virtual healthcare clinic in U.S. for common ailments\nSkip to main content\nExclusive news, data and analytics for financial market professionals\nLearn more about\nReuters home\nWorld\nBusiness\nMarkets\nBreakingviews\nLegal\nTechnology\nInvestigations\nSports\nMore\nMy View\nRegister\nEmployee Benefits & Executive Compensation\nHealth\n1 minute read\nNovember 16, 2022\n5:54 AM UTC\nLast Updated  ago\nRPT Amazon launches virtual healthcare clinic in U.S. for common ailments\nReuters\nThe Amazon logo is seen outside its JFK8 distribution center in Staten Island, New York, U.S. November 25, 2020.  REUTERS/Brendan McDermid\nCompanies\nAmazon.com Inc\nFollow\n1Life Healthcare Inc\nFollow\nNov 15 (Reuters) - Amazon.com Inc\n(AMZN.O)\non Tuesday launched Amazon clinic, a virtual platform where users can connect with healthcare providers to help treat common ailments like allergies and skin conditions.\nAmazon has for years sought to expand its presence in healthcare, w

### Main class

The main indexer logic is here. We organize it as a crawler generator that adds certain visited pages to inverted index and saves them on disk. 

- `crawl_generator_for_index` method crawles the given website doing BFS, starting from `source` within given `depth`. Considers only inner pages (starting with https://www.reuters.com/...) for visiting. To speed up, do not consider pages with content type other than `html`: `.pdf`, `.mp3`, `.avi`, `.mp4`, `.txt`, ... . If crawler encounters an article page (of a form https://www.reuters.com/TOPIC/NAME...), it saves its content in a file in `collection_path` folder and populates the inverted index calling `index_doc` method. When done, saves on disk three resulting dictionaries:
    - `doc_urls`: `{doc_id : url}`
    - `index`: `{term : [collection_frequency, (doc_id_1, doc_freq_1), (doc_id_2, doc_freq_2), ...]}`
    - `doc_lengths`: `{doc_id : doc_length}` 

    `limit` parameter is given for testing - if not `None`, break the loop when number of saved articles exceeds the `limit` and return without writing dictionaries to disk.
    
- `index_doc` method parses and preprocesses the content of a `doc` and adds it to the inverted index. Also keeps track of document lengths in a `doc_lengths` dictionary.


**TODO**: at line 38 we have a regular expression that matches for https://www.reuters.com/TOPIC/ARTICLE-NAME links, but fails for topic pages and external websites. Please write this regular expression.

In [ ]:
re.match(re.compile("\s(https:\/\/www.reuters.com\/)([A-Z]+)\/([A-Z\-]+)\s"),  "https://www.reuters.com/TOPIC/NAME-AB") 

In [ ]:
from collections import Counter
from queue import Queue
import pickle
import os
import re

class ReutersSpecificIndexer:

    def __init__(self):      
        # dictionaries to populate
        self.doc_urls = {}        
        self.index = {}
        self.doc_lengths = {}
        # preprocessor
        self.prep = Preprocessor()
        
    
    def crawl_generator_for_index(self, source, depth, collection_path="collection", limit=None):        
        #TODO generate url-s for visiting
        
        q = Queue()
        q.put((source, 0))
        visited = set()
        doc_counter = 0
        # creating a folder if needed
        if not os.path.exists(collection_path):
            os.makedirs(collection_path)
        # doing a BFS
        while not q.empty():
            url, url_depth = q.get()
            if url not in visited:
                visited.add(url)
                try:
                    doc = HtmlReutersArticle(url)    # download and parse url
                    if doc.download():
                        doc.parse()
                        # TODO PLEASE WRITE A CORRECT REGULAR EXPRESSION
                        # re.compile()
                        if re.match("https:\//www\.reuters\.com/\w+/[\w-]+/$", url):#re.compile("((https://www.reuters.com/)([A-Z]+)/([A-Z])+\-([A-Z])+)"), url):
                            doc.persist(collection_path)
                            self.doc_urls[doc_counter] = url
                            self.index_doc(doc, doc_counter)
                            doc_counter += 1                        
                            yield doc
                            if limit is not None and doc_counter == limit:
                                return
                            
                            # filter links, consider only inner html pages
                        if url_depth + 1 < depth:
                            valid_anchors = filter(self.accepted_url, doc.anchors)
                            for a in valid_anchors:
                                q.put((a[1], url_depth + 1))
    
                except FileNotFoundError as e:
                    print("Analyzing", url, "led to FileNotFoundError")
                    
    
    def accepted_url(self, anchor):
        url = str(anchor[1])
        if not url.startswith("https://www.reuters.com/"):
            return False
        if url[-4:]  in ('.pdf', '.mp3', '.avi', '.mp4', '.txt'):
            return False
        return True
        
        
    def index_doc(self, doc, doc_id):
        # add documents to index
        doc.parse()
        # preprocess - tokenize, remove stopwords and non-alphanumeric tokens and stem
        content = self.prep.preprocess(doc.article_text)
        self.doc_lengths[doc_id] = len(content)
        # get dict of terms in current article
        article_index = Counter(content)
        # update global index
        for term in article_index.keys():
            article_freq = article_index[term]
            if term not in self.index:                
                self.index[term] = [article_freq, (doc_id, article_freq)]
            else:
                self.index[term][0] += article_freq
                self.index[term].append((doc_id, article_freq))

### Tests

Please make sure your crawler prints out urls with `print(k, c.url)`

In [ ]:
indexer = ReutersSpecificIndexer()
k = 1
for c in indexer.crawl_generator_for_index(
        source="https://www.reuters.com/technology", 
        depth=2, 
        collection_path="test_collection", 
        limit=15):
    print(k, c.url)
    k += 1

assert type(indexer.index) is dict
assert type(indexer.index['reuter']) is list
assert type(indexer.index['reuter'][0]) is int
assert type(indexer.index['reuter'][1]) is tuple

<ipython-input-7-f026656eddb7>:67: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  texts = header.findAll(text=True) + content.findAll(text=True)


1 https://www.reuters.com/technology/twitter-remove-inactive-accounts-2023-05-08/
2 https://www.reuters.com/legal/apple-loses-bid-revive-us-copyright-claims-over-ios-simulation-2023-05-08/
3 https://www.reuters.com/technology/truecaller-aims-help-whatsapp-users-combat-spam-2023-05-08/
4 https://www.reuters.com/technology/disrupted/
5 https://www.reuters.com/technology/siemens-sap-say-eu-draft-data-act-puts-trade-secrets-risk-2023-05-07/
6 https://www.reuters.com/legal/twitter-beats-disabled-workers-lawsuit-over-layoffs-now-2023-05-08/
7 https://www.reuters.com/legal/litigation/
8 https://www.reuters.com/technology/crypto-stocks-fall-after-binance-halts-bitcoin-withdrawals-hours-2023-05-08/
9 https://www.reuters.com/technology/liechtenstein-allow-payment-certain-state-services-bitcoin-handelsblatt-2023-05-08/
10 https://www.reuters.com/technology/binance-again-temporarily-closes-bitcoin-withdrawals-2023-05-08/
11 https://www.reuters.com/technology/ai-pioneer-says-its-threat-world-may-be

Please test these documents contain a desired stem (or its derivate):

In [ ]:
some_stem = 'bank'
print(indexer.index[some_stem])
for pair in indexer.index[some_stem][1:]:
    print(indexer.doc_urls[pair[0]])

[2, (6, 1), (11, 1)]
https://www.reuters.com/legal/litigation/
https://www.reuters.com/technology/alibaba-logistics-arm-eyes-up-2-billion-hong-kong-ipo-sources-2023-05-08/


### 1.2.4. Building an index

In [ ]:
indexer = ReutersSpecificIndexer()
for k, c in enumerate(
                indexer
                    .crawl_generator_for_index(
                        "https://www.reuters.com/", 
                        3, 
                        "docs_collection",
                        limit=100   # optional limit
                    )):
    print(k + 1, c.url)

<ipython-input-7-f026656eddb7>:67: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  texts = header.findAll(text=True) + content.findAll(text=True)


1 https://www.reuters.com/business/yellen-is-calling-ceos-personally-warn-us-debt-ceiling-sources-2023-05-08/
2 https://www.reuters.com/technology/twitter-remove-inactive-accounts-2023-05-08/
3 https://www.reuters.com/legal/woman-who-sued-over-subway-tuna-seeks-quit-case-subway-demands-sanctions-2023-05-08/
4 https://www.reuters.com/legal/litigation/
5 https://www.reuters.com/legal/twitter-beats-disabled-workers-lawsuit-over-layoffs-now-2023-05-08/
6 https://www.reuters.com/markets/macromatters/
7 https://www.reuters.com/business/warren-buffetts-rival-attention-berkshire-weekend-squishmallows-2023-05-08/
8 https://www.reuters.com/podcasts/assad-cold-ukraines-religious-war-2023-05-08/
9 https://www.reuters.com/world/us/
10 https://www.reuters.com/world/europe/
11 https://www.reuters.com/world/uk/
12 https://www.reuters.com/world/americas/
13 https://www.reuters.com/world/middle-east/
14 https://www.reuters.com/world/top-biden-aide-discusses-yemen-peace-efforts-with-saudi-crown-prince-wh

### Index statistics

Load the index and print the statistics. May you use this data to update stopwords list?

In [ ]:
print('Total index keys count', len(indexer.index))

print('\nTop stems by number of documents they apperared in:')
sorted_by_n_docs = sorted(indexer.index.items(), key=lambda kv: (len(kv[1]), kv[0]), reverse=True)
print([(sorted_by_n_docs[i][0], len(sorted_by_n_docs[i][1])) for i in range(20)])

print('\nTop stems by overall frequency:')
sorted_by_freq = sorted(indexer.index.items(), key=lambda kv: (kv[1][0], kv[0]), reverse=True)
print([(sorted_by_freq[i][0], sorted_by_freq[i][1][0]) for i in range(20)])

Total index keys count 4902

Top stems by number of documents they apperared in:
[('your', 79), ('worldwid', 79), ('world', 79), ('westlaw', 79), ('view', 79), ('via', 79), ('use', 79), ('us', 79), ('uncov', 79), ('thomson', 79), ('termin', 79), ('term', 79), ('technolog', 79), ('tax', 79), ('strongest', 79), ('sport', 79), ('solut', 79), ('site', 79), ('see', 79), ('screen', 79)]

Top stems by overall frequency:
[('new', 1945), ('open', 1872), ('tab', 1850), ('reuter', 1220), ('categori', 687), ('may', 584), ('news', 523), ('market', 490), ('utc', 480), ('busi', 436), ('world', 435), ('data', 379), ('pm', 336), ('refinitiv', 327), ('technolog', 321), ('breakingview', 310), ('more', 307), ('about', 301), ('industri', 297), ('s', 284)]


## Answering a query (finally)

Now, given that we already have built the inverted index, it's time to utilize it for answering user queries. In this class there are two methods you need to implement:
- `boolean_retrieval`, the simplest form of document retrieval which returns a set of documents such that each one contains all query terms. Returns a set of document ids. Refer to *ch.1* of the book for details;
- `okapi_scoring`, Okapi BM25 ranking function - assigns scores to documents in the collection that are relevant to the user query. Returns a dictionary of scores, `doc_id:score`. Read about it in [Wikipedia](https://en.wikipedia.org/wiki/Okapi_BM25#The_ranking_function) and implement accordingly.

Both methods accept `query` parameter in a form of a dictionary, `term:frequency`

In [ ]:
from collections import Counter
import math

class QueryProcessing:
    
    @staticmethod
    def prepare_query(raw_query):
        prep = Preprocessor()
        # pre-process query the same way as documents
        query = prep.preprocess(raw_query)
        # count frequency
        return Counter(query)
    
    @staticmethod
    def boolean_retrieval(query, index):
        # retrieve a set of documents containing all query terms
        
        docs = []
        for q in query.keys():
          if q in index:
            docs += [[doc_id for doc_id, _ in index[q][1:]]]

        common_docs = set(docs[0])
        for d in docs[1:]:
          common_docs = common_docs.intersection(d)
        return common_docs

    @staticmethod
    def okapi_scoring(query, doc_lengths, index, k1=1.2, b=0.75):
        #TODO retrieve relevant documents with scores
        
        # TODO YOUR CODE HERE
        
        scores = {}
        N = len(doc_lengths)
        avgdl = sum(doc_lengths.values()) / float(len(doc_lengths))
        for term in query.keys():
            if term not in index:  # ignoring absent terms
                continue
            n_docs = len(index[term]) - 1
            idf = math.log10((N - n_docs + 0.5) / (n_docs + 0.5))
            postings = index[term][1:]
            for posting in postings:
                doc_id = posting[0]
                doc_tf = posting[1]
                score = idf * doc_tf * (k1 + 1) / (doc_tf + k1 * (1 - b + b * (doc_lengths[doc_id] / avgdl)))
                if doc_id not in scores:
                    scores[doc_id] = score
                else:  # accumulate scores
                    scores[doc_id] += score
        return scores

    # def okapi_scoring(query, doc_lengths, index, k1=1.2, b=0.75):
    #     #TODO retrieve relevant documents with scores
        
    #     # TODO YOUR CODE HERE
        
    #     return {0: 0.32, 5: 1.17}



### Tests

In [ ]:
test_doc_lengths = {1: 20, 2: 15, 3: 10, 4:20, 5:30}
test_index = {'x': [2, (1, 1), (2, 1)], 'y': [2, (1, 1), (3, 1)], 'z': [3, (2, 1), (4,2)]}


test_query1 = QueryProcessing.prepare_query('x z')
test_query2 = QueryProcessing.prepare_query('x y')


assert QueryProcessing.boolean_retrieval(test_query1, test_index) == {2}
assert QueryProcessing.boolean_retrieval(test_query2, test_index) == {1}
okapi_res = QueryProcessing.okapi_scoring(test_query2, test_doc_lengths, test_index)
assert all(k in okapi_res for k in (1, 2, 3))
assert not any(k in okapi_res for k in (4, 5))
assert okapi_res[1] > okapi_res[3] > okapi_res[2]

### And now query the real index

In [ ]:
import time

queries = ["russia ukraine", "Hong Kong", "crypto money"]
for q in queries:
    print(q)
    qobj = QueryProcessing.prepare_query(q)
    for res in QueryProcessing.boolean_retrieval(qobj, indexer.index):
        print('\t', indexer.doc_urls[res])

    s = time.time()
    okapi_res = QueryProcessing.okapi_scoring(qobj, indexer.doc_lengths, indexer.index)
    e = time.time()
    print(f"\t == Okapi Time: {e - s:.5f} ==")
    for res in okapi_res:
        print('\t', indexer.doc_urls[res], okapi_res[res])
    
    print()

russia ukraine
	 https://www.reuters.com/sports/chinese-premier-urges-ioc-oppose-politicisation-sports-2023-05-08/
	 https://www.reuters.com/breakingviews/joined-up-g7-is-best-china-deterrent-2023-05-08/
	 https://www.reuters.com/podcasts/assad-cold-ukraines-religious-war-2023-05-08/
	 https://www.reuters.com/world/europe/
	 https://www.reuters.com/lifestyle/eurovision-2023-opens-with-acts-walking-liverpools-turquoise-carpet-2023-05-07/
	 https://www.reuters.com/breakingviews/unicredits-ma-hunt-has-more-glitter-than-gold-2023-05-03/
	 == Okapi Time: 0.00011 ==
	 https://www.reuters.com/podcasts/assad-cold-ukraines-religious-war-2023-05-08/ 3.152651851710943
	 https://www.reuters.com/world/europe/ 3.5764576972858277
	 https://www.reuters.com/business/environment/ 1.3888763893719798
	 https://www.reuters.com/lifestyle/eurovision-2023-opens-with-acts-walking-liverpools-turquoise-carpet-2023-05-07/ 2.510973798585008
	 https://www.reuters.com/breakingviews/joined-up-g7-is-best-china-deterre